# Detección de patologías mediante ontologías y modelos del lenguaje

## Librerías

In [ ]:
!pip install deep_translator
!pip install kagglehub[hf-datasets]
!pip install keras_hub

import keras
import keras_hub
import numpy as np
import kagglehub
from kagglehub import KaggleDatasetAdapter
import numpy as np
import pandas as pd
import re
from difflib import SequenceMatcher
from deep_translator import GoogleTranslator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import precision_score, f1_score
import os
import random
import tensorflow as tf

re_simbolos = r'[^a-zA-ZáéíóúüñÁÉÍÓÚÜÑ\s]'

SEED = 1
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.8 MB/s eta 0:00:00


## Preprocesado con ontologías

Este apartado está dedicado a realizar el enriquecimiento del dataset empleado mediante las ontologías empleadas en el artículo: ICD10 y SNOMED.

Sin embargo, en mi caso no ha sido posible acceder a dichas ontologías mediante la biblioteca pymedtermino (como hacen los autores del artículo), ya que se necesitan tener descargados dichas ontologías, pero estas no se encuentran accesibles de manera pública.

Para solucionar esto, realizaremos un procedimiento semi-automático, donde relacionaremos las enfermedades del nuevo dataset con las de DermatES para enriquecerlo, y, para el resto de enfermedades tendremos que realizar la extracción a mano, ya que existen browsers online sobre dichas ontologías que si son accesibles de forma pública.

In [ ]:
file_path = "Skin_text_classifier.csv"

skin_dataset = kagglehub.load_dataset(
  KaggleDatasetAdapter.HUGGING_FACE,
  "rafsunahmad/skin-disease-text-classification",
  file_path
)

df_skin = pd.DataFrame(skin_dataset)
df_dermates = pd.read_csv("hf://datasets/fundacionctic/DermatES/datos_finales.tsv", sep="\t")

<ipython-input-2-8d443a73dc9d>:3: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  skin_dataset = kagglehub.load_dataset(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Observando ambos datasets, el objetivo es relacionar la columna Disease name de skin_text_classifier con la columna patologia de DermatES.

Relacionar ambas columnas es algo complejo, ya que en los dataset pueden aparecer las mismas enfermedades con nombres ligeramente diferentes, al estar ambos datasets con nombres médicos pero en diferentes idiomas

In [ ]:
df_skin.head()

,Disease name,Text
0,Vitiligo,"""I've had these light patches on my neck and f..."
1,Scabies,"""Doctor, I've noticed these small, red bumps o..."
2,Vitiligo,"""Doctor, I noticed a pale patch around my knee..."
3,Hives (Urticaria),"Hives, also known as urticaria, typically pres..."
4,Folliculitis,"""I have these small, hard bumps on my buttocks..."


In [ ]:
df_dermates.head()

,patologia,texto,gravedad,sitio,tipo
0,Alopecia areata,consulta telefonica no consigo hablar por tele...,0,head,autoimmune process
1,Queratosis seborreica sai,exploro las leuiones y son todas q seborreicas...,0,sun,non-cancer tumor
2,Psoriasis sai,paciente citado presencialmente que solicita c...,0,extrem,autoimmune
3,Dermatitis de contacto sai (ver eccema de cont...,paciente de años derivada desde dermatologia p...,0,hand,disease or syndrome
4,Melanoma sai,beg no sintomas sistemicos,3,any,neoplastic process


Para poder relacionar ambas columnas, realizaremos una serie de transformaciones sobre los datos.

* Eliminamos las tildes
* Pasamos a minúsculas
* Eliminamos el contenido entre paréntesis
* Eliminamos cualquier símbolo que no sean letras

In [ ]:
import unicodedata

def quitar_tildes(texto):
    return ''.join(
        c for c in unicodedata.normalize('NFD', texto)
        if unicodedata.category(c) != 'Mn'
    )

df_dermates["patologia"] = df_dermates["patologia"].apply(
    lambda x: re.sub(r'\([^()]*\)', '', x)
)

df_dermates["patologia"] = df_dermates["patologia"].apply(
    lambda x: re.sub(re_simbolos, '', x.lower())
)

df_dermates["patologia"] = df_dermates["patologia"].apply(
quitar_tildes
)

df_dermates["patologia"]

,patologia
0,alopecia areata
1,queratosis seborreica sai
2,psoriasis sai
3,dermatitis de contacto sai
4,melanoma sai
...,...
8876,carcinoma basocelular
8877,dermatitis psoriasiforme sai
8878,nevus melanocitico adquirido
8879,queratosis seborreica sai


Como procesado adicional, eliminaremos la palabra 'sai' del dataset DermatES, ya que 'sai' indica 'sin otra indicación', siendo esta versión de las patologías la más acertada para el dataset skin_disease ya que no tienen indicaciones adicionales sobre la enfermedad.

In [ ]:
df_dermates["patologia"] = df_dermates["patologia"].apply(
    lambda x: re.sub(r'\bsai\b', '', x)
)
df_dermates["patologia"] = df_dermates["patologia"].apply(lambda x: x.strip())
df_dermates["patologia"]

,patologia
0,alopecia areata
1,queratosis seborreica
2,psoriasis
3,dermatitis de contacto
4,melanoma
...,...
8876,carcinoma basocelular
8877,dermatitis psoriasiforme
8878,nevus melanocitico adquirido
8879,queratosis seborreica


Tras haber procesado ambas columnas, extraemos los valores únicos de cada una de ellas.

In [ ]:
patologias_dermat_es = df_dermates["patologia"].unique()
patologias_skin = df_skin["Disease name"].unique()

In [ ]:
patologias_skin = list(map(str.lower, patologias_skin))
patologias_skin = list(map(lambda x: re.sub(re_simbolos, '', x.lower()), patologias_skin))
patologias_skin

['vitiligo',
 'scabies',
 'hives urticaria',
 'folliculitis',
 'eczema',
 'ringworm tinea corporis',
 'athletes foot tinea pedis',
 'rosacea',
 'psoriasis',
 'shingles herpes zoster',
 'impetigo',
 'contact dermatitis',
 'acne']

Como primera aproximación, utilizaremos la función de similitud de SequenceMatcher, que nos da un valor entre 1.0 y 0.0 indicando como de parecido son dos términos en base a sus caracteres.

Utilizaremos los 3 términos más cercanos con los términos en inglés originales del dataset skin_disease.

Tras ejecutarlo, podemos ver que en los casos: urticaria, foliculitis, rosacea, psoriasis, herpes zoster y acne, el términos más cercano coincide.

In [ ]:
#probamos con los términos del dataset skin disease en inglés

def terminos_parecidos(lterm1, lterm2, k=3):
    def sim(a, b): return SequenceMatcher(None, a, b).ratio()
    res = {}
    for t1 in lterm1:
        l = [(t2,sim(t1,t2)) for t2 in lterm2]
        l.sort(reverse=True, key=lambda x: x[1])
        res[t1] = l[:k]
    return res

dict_parecidos = terminos_parecidos(patologias_skin, patologias_dermat_es)
dict_parecidos

{'vitiligo': [('lentigo', 0.5333333333333333),
  ('lentigo maligno', 0.5217391304347826),
  ('vasculitis livedoide', 0.5)],
 'scabies': [('psoriasis', 0.5),
  ('vasculitis', 0.47058823529411764),
  ('rosacea', 0.42857142857142855)],
 'hives urticaria': [('urticaria', 0.75),
  ('urticaria aguda', 0.6),
  ('urticaria solar', 0.6)],
 'folliculitis': [('foliculitis', 0.9565217391304348),
  ('foliculitis aguda', 0.7586206896551724),
  ('foliculitis cronica', 0.7096774193548387)],
 'eczema': [('eczema', 1.0),
  ('eccema', 0.8333333333333334),
  ('eritema', 0.6153846153846154)],
 'ringworm tinea corporis': [('carcinoma espinocelular in situ',
   0.48148148148148145),
  ('carcinoma espinocelular', 0.4782608695652174),
  ('carcinoma verrucoso', 0.47619047619047616)],
 'athletes foot tinea pedis': [('herpes gestationis', 0.5116279069767442),
  ('dermatitis fotoalergica a drogas', 0.45614035087719296),
  ('alopecia difusa', 0.45)],
 'rosacea': [('rosacea', 1.0),
  ('acne rosacea', 0.7368421052631

Almacenamos los términos de skin_disease que son los correspondientes en español en DermatES.

In [ ]:
relacionados = {
    "hives urticaria": dict_parecidos["hives urticaria"][0][0],
    "rosacea": dict_parecidos["rosacea"][0][0],
    "acne": dict_parecidos["acne"][0][0],
    "shingles herpes zoster": dict_parecidos["shingles herpes zoster"][0][0],
    "eczema": dict_parecidos["eczema"][0][0],
    "folliculitis": dict_parecidos['folliculitis'][0][0],
    "psoriasis": dict_parecidos["psoriasis"][0][0]
}
relacionados

{'hives urticaria': 'urticaria',
 'rosacea': 'rosacea',
 'acne': 'acne',
 'shingles herpes zoster': 'herpes zoster',
 'eczema': 'eczema',
 'folliculitis': 'foliculitis',
 'psoriasis': 'psoriasis'}

Por el momento, nos quedan un total de 6 enfermedades por relacionar con el dataset dermatES.

In [ ]:
restantes = [nombre for nombre,_ in dict_parecidos.items() if nombre not in relacionados.keys()]
restantes

['vitiligo',
 'scabies',
 'ringworm tinea corporis',
 'athletes foot tinea pedis',
 'impetigo',
 'contact dermatitis']

Como último intento, traduciremos los términos restantes al español y volveremos a buscar los 3 términos más cercanos.

In [ ]:
restantes_es = [GoogleTranslator(source='en', target='es').translate(p) for p in restantes]
restantes_es = list(map(quitar_tildes, restantes_es))

Podemos observar que solo hemos obtenido una coincidencia en dermatitis de contacto, por lo que nos quedan un total de 5 términos que no se encuentran en el dataset dermatES.

In [ ]:
terminos_parecidos(restantes_es, patologias_dermat_es)

{'vitiligo': [('lentigo', 0.5333333333333333),
  ('lentigo maligno', 0.5217391304347826),
  ('vasculitis livedoide', 0.5)],
 'sarna': [('carcinoma', 0.5714285714285714),
  ('rosacea', 0.5),
  ('angioma', 0.5)],
 'tina de tinea corpor': [('pitiriasis versicolor', 0.4878048780487805),
  ('nevus del tejido conectivo', 0.4782608695652174),
  ('acne excoriado', 0.47058823529411764)],
 'atletas pie tinea pedis': [('alopecia difusa', 0.47368421052631576),
  ('alopecia androgena inducida por drogas', 0.45901639344262296),
  ('dermatitis pruriginosa', 0.4444444444444444)],
 'impetigo': [('lentigo', 0.6666666666666666),
  ('penfigoide', 0.5555555555555556),
  ('eccema impetiginizado', 0.5517241379310345)],
 'dermatitis de contacto': [('dermatitis de contacto', 1.0),
  ('dermatitis del panal', 0.7619047619047619),
  ('eccema de contacto', 0.75)]}

In [ ]:
relacionados["contact dermatitis"] = "dermatitis de contacto"
restantes.remove("contact dermatitis")

In [ ]:
restantes_dict = {k:None for k in restantes}
restantes_dict

{'vitiligo': None,
 'scabies': None,
 'ringworm tinea corporis': None,
 'athletes foot tinea pedis': None,
 'impetigo': None}

In [ ]:
df_dermates["sitio"].unique(), df_dermates["tipo"].unique()

(array(['head', 'sun', 'extrem', 'hand', 'any', 'joint', 'face', 'chest',
        'connective tissue', 'mouth', 'sex', 'leg'], dtype=object),
 array(['autoimmune process', 'non-cancer tumor', 'autoimmune',
        'disease or syndrome', 'neoplastic process', 'preneoplastic',
        'abnormality', 'infectious process', 'symptom', 'syndrome',
        'infection', 'qualitative concept', 'pathologic function',
        'injury or poisoning'], dtype=object))

Con todo esto, como las enfermedades restantes no se encuentran en el dataset DermatES, necesitamos realizar la extracción de las características de las enfermedades de manera manual utilizando los browsers públicos de las ontologías.

Ontologías:

SNOMED CT: https://browser.ihtsdotools.org/?

ICD-10: https://www.icd10data.com/

In [ ]:
restantes_dict["scabies"] = {
    "location": "any",
    "type": "parasitic process",
    "severity": 1
}

restantes_dict["impetigo"] = {
    "location": "face",
    "type": "infectious process",
    "severity": 0
}

restantes_dict["ringworm tinea corporis"] = {
    "location": "any",
    "type": "infectious process",
    "severity": 1
}

restantes_dict["vitiligo"] = {
    "location": "face",
    "type": "hypopigmentation",
    "severity": 1
}

restantes_dict["athletes foot tinea pedis"] = {
    "location": "foot",
    "type": "infectious process",
    "severity": 1
}

In [ ]:
restantes_dict

{'vitiligo': {'location': 'face', 'type': 'hypopigmentation', 'severity': 1},
 'scabies': {'location': 'any', 'type': 'parasitic process', 'severity': 1},
 'ringworm tinea corporis': {'location': 'any',
  'type': 'infectious process',
  'severity': 1},
 'athletes foot tinea pedis': {'location': 'foot',
  'type': 'infectious process',
  'severity': 1},
 'impetigo': {'location': 'face', 'type': 'infectious process', 'severity': 0}}

Para el resto de términos que relacionamos, almacenamos la información correspondiente utilizando los datos en DermatES.

In [ ]:
for k, v in relacionados.items():
    row = df_dermates[df_dermates["patologia"] == v].iloc[0]
    restantes_dict[k] = {
        "location": row["sitio"],
        "type": row["tipo"],
        "severity": row["gravedad"]
    }

Con todo esto, ya tenemos finalmente cada enfermedad enriquecida con la información de las ontologías.

In [ ]:
dict_final = dict(restantes_dict)
dict_final

{'vitiligo': {'location': 'face', 'type': 'hypopigmentation', 'severity': 1},
 'scabies': {'location': 'any', 'type': 'parasitic process', 'severity': 1},
 'ringworm tinea corporis': {'location': 'any',
  'type': 'infectious process',
  'severity': 1},
 'athletes foot tinea pedis': {'location': 'foot',
  'type': 'infectious process',
  'severity': 1},
 'impetigo': {'location': 'face', 'type': 'infectious process', 'severity': 0},
 'hives urticaria': {'location': 'any',
  'type': 'pathologic function',
  'severity': np.int64(1)},
 'rosacea': {'location': 'face',
  'type': 'disease or syndrome',
  'severity': np.int64(0)},
 'acne': {'location': 'any',
  'type': 'disease or syndrome',
  'severity': np.int64(1)},
 'shingles herpes zoster': {'location': 'chest',
  'type': 'infection',
  'severity': np.int64(2)},
 'eczema': {'location': 'hand',
  'type': 'disease or syndrome',
  'severity': np.int64(0)},
 'folliculitis': {'location': 'head',
  'type': 'disease or syndrome',
  'severity': np.

### Generación Dataset Final

In [ ]:
df_skin_final = pd.DataFrame()
df_skin_temp = pd.DataFrame(df_skin)

df_skin_temp["Disease name"] = df_skin_temp["Disease name"].apply(str.lower)
df_skin_temp["Disease name"] = df_skin_temp["Disease name"].apply(lambda x: re.sub(re_simbolos, '', x.lower()))


new_rows = []

for index,row in df_skin_temp.iterrows():
    new_row = {
        "Disease name": row["Disease name"],
        "Text": row["Text"],
        "Location": dict_final[row["Disease name"]]["location"],
        "Type": dict_final[row["Disease name"]]["type"],
        "Severity": dict_final[row["Disease name"]]["severity"],
    }
    new_rows.append(new_row)

df_skin_final = pd.DataFrame(new_rows)
df_skin_final.head()

,Disease name,Text,Location,Type,Severity
0,vitiligo,"""I've had these light patches on my neck and f...",face,hypopigmentation,1
1,scabies,"""Doctor, I've noticed these small, red bumps o...",any,parasitic process,1
2,vitiligo,"""Doctor, I noticed a pale patch around my knee...",face,hypopigmentation,1
3,hives urticaria,"Hives, also known as urticaria, typically pres...",any,pathologic function,1
4,folliculitis,"""I have these small, hard bumps on my buttocks...",head,disease or syndrome,0


In [ ]:
df_skin_final.to_csv("skin_final.csv",index=False)

### Partición en train validation y test

In [ ]:
X = df_skin_final[["Text","Location","Type","Severity"]]
y = df_skin_final["Disease name"]

encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

X_temp_raw, X_test_raw, y_temp, y_test = train_test_split(
    X, y_encoded,
    test_size=0.15,
    random_state=42,
    stratify=y_encoded
)

X_train_raw, X_val_raw, y_train, y_val = train_test_split(
    X_temp_raw, y_temp,
    test_size=0.1765,
    random_state=42,
    stratify=y_temp
)

## Entrenamiento de transformers

### Preprocesado para Keras

Para poder trabajar con Keras necesitamos transformar la variable objetivo de cada modelo en un vector o valor numérico que represente la clase a la que pertenece. Para ello podemos utilizar LabelEncoder.

Además de transformar la variable objetivo, necesitamos tokenizar el texto.

En este caso, como emplearemos un total de cuatro modelos para clasificar el tipo de patología, la localización, la severidad y la enfermedad, tokenizaremos cada información y la almacenaremos en columnas con otro nombre.

De esta manera:
* El clasificador del tipo de patología recibe solo la columna "Text", que es la descripción.
* El clasificador de la localización recibe la columna "Text + Type", descripción y tipo de patología.
* El clasificador de la severidad recibe "Text + Type + Location", de manera análoga a los anteriores.
* Finalmente, el clasificador de la enfermedad recibe "Text + Type + Location + Severity"

In [ ]:
encoder_type = LabelEncoder()
encoder_location = LabelEncoder()
encoder_severity = LabelEncoder()
encoder_type.fit(X["Type"])
encoder_location.fit(X["Location"])
encoder_severity.fit(X["Severity"])

y_train_encoded_type = encoder_type.transform(X_train_raw["Type"])
y_train_encoded_location = encoder_location.transform(X_train_raw["Location"])
y_train_encoded_severity = encoder_severity.transform(X_train_raw["Severity"])

y_val_encoded_type = encoder_type.transform(X_val_raw["Type"])
y_val_encoded_location = encoder_location.transform(X_val_raw["Location"])
y_val_encoded_severity = encoder_severity.transform(X_val_raw["Severity"])

y_test_encoded_type = encoder_type.transform(X_test_raw["Type"])
y_test_encoded_location = encoder_location.transform(X_test_raw["Location"])
y_test_encoded_severity = encoder_severity.transform(X_test_raw["Severity"])


X_train_raw["Text + Type"] = X_train_raw["Text"] + " Type: " + X_train_raw["Type"]
X_val_raw["Text + Type"] = X_val_raw["Text"] + " Type: " + X_val_raw["Type"]
X_test_raw["Text + Type"] = X_test_raw["Text"] + " Type: " + X_test_raw["Type"]

X_train_raw["Text + Type + Location"] = X_train_raw["Text + Type"] + " Location: " + X_train_raw["Location"]
X_val_raw["Text + Type + Location"] = X_val_raw["Text + Type"] + " Location: " + X_val_raw["Location"]
X_test_raw["Text + Type + Location"] = X_test_raw["Text + Type"] + " Location: " + X_test_raw["Location"]

X_train_raw["Text + Type + Location + Severity"] = X_train_raw["Text + Type + Location"] + " Severity: " + X_train_raw["Severity"].apply(str)
X_val_raw["Text + Type + Location + Severity"] = X_val_raw["Text + Type + Location"] + " Severity: " + X_val_raw["Severity"].apply(str)
X_test_raw["Text + Type + Location + Severity"] = X_test_raw["Text + Type + Location"] + " Severity: " + X_test_raw["Severity"].apply(str)

### Creación de modelos

El modelo elegido es bert_small_en_uncased, un modelo basado en BERT, entrenado con un corpus en inglés y que trabaja solo con minúsculas.

Fijaremos el tamaño máximo de secuencia en 256 tokens para acelerar el proceso de entrenamiento (este valor permite no realizar truncamiento y es potencia de 2).

Se ha elegido realizar un full fine tuning ya que el rendimiento de este modelo se ve muy afectado al no poder entrenar el backbone.

Tenemos un total de 4 modelos de clasificación.

El primer modelo: classifier_type intenta clasificar el tipo de patología de la enfermedad en base a la descripción de los síntomas.

El segundo modelo: classifier_location intenta clasificar la localización (en el cuerpo) de la patología utilizando tanto la descripción de los síntomas como el tipo de patología.

El tercer modelo: classifier_severity intenta clasificar el grado de severidad (valor entre 0 y 3) utilizando la descripción de los síntomas, el tipo de patología y la localización de la enfermedad.

El cuarto modelo y final: classifier_disease intenta clasificar la enfermedad utilizando todos los datos anteriores. En la arquitectura en cascada, partiremos de la descripción e iremos enriqueciendo dicha descripción agregando la predicción de cada uno de los modelos anteriores.

In [ ]:
preprocessor = keras_hub.models.BertPreprocessor.from_preset("bert_small_en_uncased", sequence_length=256)

classifier_type = keras_hub.models.BertClassifier.from_preset(
    "bert_small_en_uncased",
    num_classes=len(encoder_type.classes_),
    preprocessor=None,
)
classifier_type.backbone.trainable = True

classifier_location = keras_hub.models.BertClassifier.from_preset(
    "bert_small_en_uncased",
    num_classes=len(encoder_location.classes_),
    preprocessor=None,
)

classifier_location.backbone.trainable = True

classifier_severity = keras_hub.models.BertClassifier.from_preset(
    "bert_small_en_uncased",
    num_classes=len(encoder_severity.classes_),
    preprocessor=None,
)

classifier_severity.backbone.trainable = True

classifier_disease = keras_hub.models.BertClassifier.from_preset(
    "bert_small_en_uncased",
    num_classes=len(encoder.classes_),
    preprocessor=None,
)

classifier_disease.backbone.trainable = True

datos = {
    "type": {
        "train": (preprocessor(X_train_raw["Text"].tolist()), y_train_encoded_type),
        "val": (preprocessor(X_val_raw["Text"].tolist()), y_val_encoded_type),
        "test": (preprocessor(X_test_raw["Text"].tolist()), y_test_encoded_type),
        "model": classifier_type,
        "encoder": encoder_type
    },
    "location": {
        "train": (preprocessor(X_train_raw["Text + Type"].tolist()), y_train_encoded_location),
        "val": (preprocessor(X_val_raw["Text + Type"].tolist()), y_val_encoded_location),
        "test": (preprocessor(X_test_raw["Text + Type"].tolist()), y_test_encoded_location),
        "model": classifier_location,
        "encoder": encoder_location
    },
    "severity": {
        "train": (preprocessor(X_train_raw["Text + Type + Location"].tolist()), y_train_encoded_severity),
        "val": (preprocessor(X_val_raw["Text + Type + Location"].tolist()), y_val_encoded_severity),
        "test": (preprocessor(X_test_raw["Text + Type + Location"].tolist()), y_test_encoded_severity),
        "model": classifier_severity,
        "encoder": encoder_severity
    },
    "disease": {
        "train": (preprocessor(X_train_raw["Text + Type + Location + Severity"].tolist()), y_train),
        "val": (preprocessor(X_val_raw["Text + Type + Location + Severity"].tolist()), y_val),
        "test": (preprocessor(X_test_raw["Text + Type + Location + Severity"].tolist()), y_test),
        "model": classifier_disease,
        "encoder": encoder
    }
}

### Entrenamiento

Entrenaremos todos los modelos bajo las mismas condiciones:
- Un máximo de 20 epochs
- Realizaremos un EarlyStopping tras dos épocas sin mejorar sobre la función de pérdida en validation
- Se utilizarán los parámetros que menor valor de función de pérdida han conseguido.

In [ ]:
def entrenar_modelo(modelo, X_train, y_train, X_val, y_val, num_classes, epochs=10, batch_size=8):
    callback = EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    )

    modelo.compile(
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer=keras.optimizers.Adam(5e-5),
        jit_compile=True,
    )
    modelo.fit(
        x=X_train,
        y=y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(X_val, y_val),
        callbacks = [callback]
    )
    return modelo

def pipeline_entrenamiento(datos):
    resultados = {}
    for key,value in datos.items():
        print("Entrenando modelo para", key)

        model = value["model"]
        X_train, y_train = value["train"]
        X_val, y_val = value["val"]
        X_test,y_test = value["test"]

        entrenar_modelo(value["model"],X_train, y_train, X_val, y_val, num_classes=len(value["encoder"].classes_), epochs=40, batch_size=16)

        train_loss, train_accuracy = model.evaluate(X_train, y_train, verbose=0)
        val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
        test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)

        y_pred_train = model.predict(X_train)
        y_pred_val = model.predict(X_val)
        y_pred_test = model.predict(X_test)

        y_pred_train_classes = np.argmax(y_pred_train, axis=1)
        y_pred_val_classes = np.argmax(y_pred_val, axis=1)
        y_pred_test_classes = np.argmax(y_pred_test, axis=1)


        train_precision = precision_score(y_train, y_pred_train_classes, average="weighted", zero_division=0)
        val_precision = precision_score(y_val, y_pred_val_classes, average="weighted", zero_division=0)
        test_precision = precision_score(y_test, y_pred_test_classes, average="weighted", zero_division=0)

        train_f1 = f1_score(y_train, y_pred_train_classes, average="weighted", zero_division=0)
        val_f1 = f1_score(y_val, y_pred_val_classes, average="weighted", zero_division=0)
        test_f1 = f1_score(y_test, y_pred_test_classes, average="weighted", zero_division=0)

        resultados[key] = {
            "train_accuracy": train_accuracy,
            "val_accuracy": val_accuracy,
            "train_precision": train_precision,
            "val_precision": val_precision,
            "train_f1": train_f1,
            "val_f1": val_f1,
            "test_accuracy": test_accuracy,
            "test_f1": test_f1,
            "test_precision": test_precision
        }

    return resultados

In [ ]:
resultados = pipeline_entrenamiento(datos)

Entrenando modelo para type
Epoch 1/40
7/7 ━━━━━━━━━━━━━━━━━━━━ 46s 3s/step - loss: 1.8327 - sparse_categorical_accuracy: 0.3403 - val_loss: 1.6748 - val_sparse_categorical_accuracy: 0.4091
Epoch 2/40
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 116ms/step - loss: 1.6041 - sparse_categorical_accuracy: 0.4447 - val_loss: 1.7389 - val_sparse_categorical_accuracy: 0.2727
Epoch 3/40
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 135ms/step - loss: 1.6027 - sparse_categorical_accuracy: 0.4263 - val_loss: 1.6563 - val_sparse_categorical_accuracy: 0.4091
Epoch 4/40
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 134ms/step - loss: 1.4609 - sparse_categorical_accuracy: 0.5028 - val_loss: 1.6269 - val_sparse_categorical_accuracy: 0.4091
Epoch 5/40
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 134ms/step - loss: 1.3602 - sparse_categorical_accuracy: 0.5451 - val_loss: 1.4650 - val_sparse_categorical_accuracy: 0.4545
Epoch 6/40
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 135ms/step - loss: 1.1942 - sparse_categorical_accuracy: 0.5999 - val_loss: 1.3887 - val_sparse_categorical_accurac

3/4 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step

4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 711ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 861ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
Entrenando modelo para disease
Epoch 1/40
7/7 ━━━━━━━━━━━━━━━━━━━━ 44s 3s/step - loss: 2.5897 - sparse_categorical_accuracy: 0.0810 - val_loss: 2.5076 - val_sparse_categorical_accuracy: 0.1364
Epoch 2/40
7/7 ━━━━━━━━━━━━━━━━━━━━ 11s 142ms/step - loss: 2.4503 - sparse_categorical_accuracy: 0.2429 - val_loss: 2.3894 - val_sparse_categorical_accuracy: 0.2727
Epoch 3/40
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 140ms/step - loss: 2.3317 - sparse_categorical_accuracy: 0.2837 - val_loss: 2.2178 - val_sparse_categorical_accuracy: 0.3182
Epoch 4/40
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 142ms/step - loss: 2.1714 - sparse_categorical_accuracy: 0.4356 - val_loss: 1.9756 - val_sparse_categorical_accuracy: 0.5000
Epoch 5/40
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 141ms/step - loss: 1.9827 - sparse_categorical_accuracy: 0.4837 - val_loss: 1.7463 - val_sparse_categorical_accuracy: 0.5455
Epoch 6/40
7/7 ━━━━━━━━━━━━━━━━━━━

## Resultados

Tras haber entrenado los modelos, podemos analizar sus rendimientos de manera separada.

En el caso del modelo para predecir el tipo de patología, nos encontramos con un modelo que parece sobreajustado al tener un valor perfecto en train pero menor en validación y todavía menor en test.

El modelo de predicción sobre la localización de la patología muestra valores más equilibrados entre validación y test, alrededro del 70% de accuracy, por lo que su rendimiento es superior al modelo de tipo.

Para los modelos de severidad y el modelo final de predicción sobre la enfermedad, observamos métricas muy buenas, alrededor de un 95% tanto en accuracy como precision y f1 ponderado.

In [ ]:
resultados

{'type': {'train_accuracy': 1.0,
  'val_accuracy': 0.7272727489471436,
  'train_precision': 1.0,
  'val_precision': 0.696969696969697,
  'train_f1': 1.0,
  'val_f1': 0.7090909090909091,
  'test_accuracy': 0.6818181872367859,
  'test_f1': 0.6439393939393939,
  'test_precision': 0.6532467532467533},
 'location': {'train_accuracy': 0.9090909361839294,
  'val_accuracy': 0.7727272510528564,
  'train_precision': 0.8487373737373738,
  'val_precision': 0.8068181818181818,
  'train_f1': 0.8743364790313146,
  'val_f1': 0.7583333333333333,
  'test_accuracy': 0.8181818127632141,
  'test_f1': 0.7741702741702742,
  'test_precision': 0.759090909090909},
 'severity': {'train_accuracy': 1.0,
  'val_accuracy': 1.0,
  'train_precision': 1.0,
  'val_precision': 1.0,
  'train_f1': 1.0,
  'val_f1': 1.0,
  'test_accuracy': 0.9090909361839294,
  'test_f1': 0.9078282828282828,
  'test_precision': 0.923076923076923},
 'disease': {'train_accuracy': 1.0,
  'val_accuracy': 1.0,
  'train_precision': 1.0,
  'val_pre

Para mostrar los resultados en el artículo, guardamos los resultados en un archivo csv mediante pandas.

In [ ]:
df_resultados = pd.DataFrame.from_dict(resultados, orient="index")
df_resultados.to_csv("resultados.csv")

Finalmente, ante una prueba en cascada real, podemos ver que el rendimiento de la arquitectura final no es especialmente bueno, con un 50% de accuracy.

Podemos ver que el rendimiento de los modelos ronda el 50% a excepción del modelo predictor de la severidad, con un 68%.

A pesar de haber obtenido en los modelos de severidad y enfermedad métricas de predicción muy altas, la prueba final en cascada demuestra: tanto la importancia del enriquecimiento mediante ontologías, como asegurar un rendimiento entre modelos en cascada parejos, ya que, aunque los dos últimos modelos han obtenido métricas muy buenas, podemos observar la merma en el rendimiento al pasar a un modo en cascada.

In [ ]:
from sklearn.metrics import accuracy_score

#primer modelo
rendimiento_final = []
X_siguiente = preprocessor(X_test_raw["Text"])

type_t = datos["type"]["model"].predict(X_siguiente)
type_t_classes = np.argmax(type_t, axis=1)
type_t_text = datos["type"]["encoder"].inverse_transform(type_t_classes)

rendimiento_final.append(
    {
        "Model": "Type",
        "Acc": datos["type"]["model"].evaluate(X_siguiente, datos["type"]["test"][1], verbose=0)[1],
        "Prec": precision_score(type_t_classes, datos["type"]["test"][1], average="weighted", zero_division=0),
        "F1": f1_score(type_t_classes, datos["type"]["test"][1], average="weighted", zero_division=0)
    }
)


X_siguiente = preprocessor(X_test_raw["Text"] + " Type: " + type_t_text)

#segundo modelo
location_t = datos["location"]["model"].predict(X_siguiente)
location_t_classes = np.argmax(location_t, axis=1)
location_t_text = datos["location"]["encoder"].inverse_transform(location_t_classes)


rendimiento_final.append(
    {
        "Model": "Location",
        "Acc": accuracy_score(location_t_classes, datos["location"]["test"][1]),
        "Prec": precision_score(location_t_classes, datos["location"]["test"][1], average="weighted", zero_division=0),
        "F1": f1_score(location_t_classes, datos["location"]["test"][1], average="weighted", zero_division=0)
    }
)


X_siguiente = preprocessor(X_test_raw["Text"] + " Type: " + type_t_text + " Location: " + location_t_text)

#tercer modelo
severity_t = datos["severity"]["model"].predict(X_siguiente)
severity_t_classes = np.argmax(severity_t, axis=1)
severity_t_text = datos["severity"]["encoder"].inverse_transform(severity_t_classes)

rendimiento_final.append(
    {
        "Model": "Severity",
        "Acc": accuracy_score(severity_t_classes, datos["severity"]["test"][1]),
        "Prec": precision_score(severity_t_classes, datos["severity"]["test"][1], average="weighted", zero_division=0),
        "F1": f1_score(severity_t_classes, datos["severity"]["test"][1], average="weighted", zero_division=0)
    }
)


X_siguiente = preprocessor(X_test_raw["Text"] + " Type: " + type_t_text + " Location: " + location_t_text + " Severity: " + [str(a) for a in severity_t_text])


#modelo final
disease_t = datos["disease"]["model"].predict(X_siguiente)
disease_t_classes = np.argmax(disease_t, axis=1)

rendimiento_final.append(
    {
        "Model": "Disease",
        "Acc": accuracy_score(disease_t_classes, datos["disease"]["test"][1]),
        "Prec": precision_score(disease_t_classes, datos["disease"]["test"][1], average="weighted", zero_division=0),
        "F1": f1_score(disease_t_classes, datos["disease"]["test"][1], average="weighted", zero_division=0)
    }
)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step


In [ ]:
pd.DataFrame(rendimiento_final).to_csv("resultados_cascada.csv",index=False)

Para preservar los parámetros e importarlos en otros archivos, podemos guardar los modelos ya entrenados.

In [ ]:
datos["severity"]["model"].save("model_severity.keras")
datos["type"]["model"].save("model_type.keras")
datos["location"]["model"].save("model_location.keras")
datos["disease"]["model"].save("model_disease.keras")